# Improving Particle Simulation with Vocabulary Types

Let's enhance our particle simulation by progressively introducing modern C++ vocabulary types. We'll start with the basic implementation and improve it step by step.

## Basic Thrust Implementation

First, let's convert our CPU code to use Thrust:



In [1]:
#Specifying path to where nvcc exists so that the jupyter notebook reads from it. nvcc is the nvidia cuda compiler for executing cuda. 
import os
os.environ['PATH'] = "/packages/apps/spack/21/opt/spack/linux-rocky8-zen3/gcc-12.1.0/cuda-12.6.1-cf4xlcbcfpwchqwo5bktxyhjagryzcx6/bin:" + os.environ['PATH']

In [2]:
%%writefile codes/basic_thrust_particles.cu
#include <thrust/universal_vector.h>
#include <thrust/transform.h>
#include <thrust/execution_policy.h>
#include <cstdio>

struct Particle {
    float x, y;    // position
    float vx, vy;  // velocity
};

int main() {
    // Simulation parameters
    float dt = 0.1f;  // time step
    
    // Create particles with initial conditions
    thrust::universal_vector<Particle> particles{
        {0.0f, 0.0f, 1.0f, 0.5f},
        {1.0f, 2.0f, -0.5f, 0.2f},
        {-1.0f, -1.0f, 0.3f, 0.7f}
    };
    
    // Define update function
    auto update_position = [dt] __host__ __device__ (Particle p) {
        p.x += p.vx * dt;
        p.y += p.vy * dt;
        return p;
    };
    
    // Print initial state
    printf("Step 0:\n");
    for (int i = 0; i < particles.size(); i++) {
        printf("Particle %d: pos=(%.2f, %.2f) vel=(%.2f, %.2f)\n", 
               i, particles[i].x, particles[i].y, particles[i].vx, particles[i].vy);
    }
    
    // Run simulation
    for (int step = 1; step <= 3; step++) {
        thrust::transform(
            thrust::device,
            particles.begin(), 
            particles.end(),
            particles.begin(),
            update_position
        );
        
        printf("\nStep %d:\n", step);
        for (int i = 0; i < particles.size(); i++) {
            printf("Particle %d: pos=(%.2f, %.2f) vel=(%.2f, %.2f)\n", 
                   i, particles[i].x, particles[i].y, particles[i].vx, particles[i].vy);
        }
    }
}

Writing codes/basic_thrust_particles.cu


In [ ]:
%%bash
nvcc -o codes/basic_pair_particles --extended-lambda codes/basic_pair_particles.cu
./codes/basic_pair_particles

## Using Pairs for Positions and Velocities

Let's improve it by using `cuda::std::pair` for position and velocity vectors:

In [4]:
%%writefile codes/pair_particles.cu
#include <thrust/universal_vector.h>
#include <thrust/transform.h>
#include <thrust/execution_policy.h>
#include <cstdio>

struct Particle {
    cuda::std::pair<float, float> pos;  // position
    cuda::std::pair<float, float> vel;  // velocity
};

// Helper function for vector addition
__host__ __device__
cuda::std::pair<float, float> operator*(const cuda::std::pair<float, float>& vec, float scalar) {
    return cuda::std::make_pair(vec.first * scalar, vec.second * scalar);
}

__host__ __device__
cuda::std::pair<float, float> operator+(
    const cuda::std::pair<float, float>& a, 
    const cuda::std::pair<float, float>& b) {
    return cuda::std::make_pair(a.first + b.first, a.second + b.second);
}

int main() {
    float dt = 0.1f;  // time step
    
    // Create particles using pairs
    thrust::universal_vector<Particle> particles{
        {cuda::std::make_pair(0.0f, 0.0f), cuda::std::make_pair(1.0f, 0.5f)},
        {cuda::std::make_pair(1.0f, 2.0f), cuda::std::make_pair(-0.5f, 0.2f)},
        {cuda::std::make_pair(-1.0f, -1.0f), cuda::std::make_pair(0.3f, 0.7f)}
    };
    
    // Update function using vector operations
    auto update_position = [dt] __host__ __device__ (Particle p) {
        p.pos = p.pos + p.vel * dt;
        return p;
    };
    
    // Print initial state
    printf("Step 0:\n");
    for (int i = 0; i < particles.size(); i++) {
        printf("Particle %d: pos=(%.2f, %.2f) vel=(%.2f, %.2f)\n", 
               i, particles[i].pos.first, particles[i].pos.second, 
               particles[i].vel.first, particles[i].vel.second);
    }
    
    // Run simulation
    for (int step = 1; step <= 3; step++) {
        thrust::transform(
            thrust::device,
            particles.begin(), 
            particles.end(),
            particles.begin(),
            update_position
        );
        
        printf("\nStep %d:\n", step);
        for (int i = 0; i < particles.size(); i++) {
            printf("Particle %d: pos=(%.2f, %.2f) vel=(%.2f, %.2f)\n", 
                   i, particles[i].pos.first, particles[i].pos.second,
                   particles[i].vel.first, particles[i].vel.second);
        }
    }
}

Writing codes/pair_particles.cu


In [ ]:
%%bash
nvcc -o codes/pair_particles --extended-lambda codes/pair_particles.cu
./codes/pair_particles

## Using mdspan for Particle Arrays

Finally, let's use `mdspan` to handle our particle data as a structured grid:

In [5]:
%%writefile codes/mdspan_particles.cu
#include <thrust/universal_vector.h>
#include <thrust/transform.h>
#include <thrust/execution_policy.h>
#include <cuda/std/mdspan>
#include <cstdio>

struct ParticleData {
    float x, y;
    float vx, vy;
};

int main() {
    float dt = 0.1f;
    const int NUM_PARTICLES = 3;
    
    // Store particle data in a single array
    thrust::universal_vector<float> particle_data{
        // x    y     vx    vy
        0.0f,  0.0f,  1.0f, 0.5f,   // Particle 0
        1.0f,  2.0f, -0.5f, 0.2f,   // Particle 1
        -1.0f, -1.0f, 0.3f, 0.7f    // Particle 2
    };
    
    // Create mdspan view of particle data
    float* data_ptr = thrust::raw_pointer_cast(particle_data.data());
    cuda::std::mdspan particles(data_ptr, NUM_PARTICLES, 4);  // 4 components per particle
    
    // Print initial state
    printf("Step 0:\n");
    for (int i = 0; i < NUM_PARTICLES; i++) {
        printf("Particle %d: pos=(%.2f, %.2f) vel=(%.2f, %.2f)\n", 
               i, particles(i,0), particles(i,1), particles(i,2), particles(i,3));
    }
    
    // Run simulation
    for (int step = 1; step <= 3; step++) {
        thrust::transform(
            thrust::device,
            particle_data.begin(), 
            particle_data.end() - 3,  // Don't transform last particle's velocity
            particle_data.begin(),
            [dt, particles] __device__ (float val) {
                int idx = &val - particles.data_handle();
                int particle_idx = idx / 4;
                int component = idx % 4;
                
                if (component < 2) {  // Position components
                    return val + dt * particles(particle_idx, component + 2);
                }
                return val;  // Velocity components unchanged
            }
        );
        
        printf("\nStep %d:\n", step);
        for (int i = 0; i < NUM_PARTICLES; i++) {
            printf("Particle %d: pos=(%.2f, %.2f) vel=(%.2f, %.2f)\n", 
                   i, particles(i,0), particles(i,1), particles(i,2), particles(i,3));
        }
    }
}

Writing codes/mdspan_particles.cu


In [ ]:
%%bash
nvcc -o codes/mdspan_particles --extended-lambda codes/mdspan_particles.cu
./codes/mdspan_particles

## Key Improvements

Each version adds benefits:

1. **Basic Thrust Version**
   - Moves computation to GPU
   - Uses thrust::transform for parallel processing

2. **Pairs Version**
   - More semantic representation of vectors
   - Vector operations are clearer
   - Better type safety

3. **mdspan Version**
   - Structured view of particle data
   - Better memory layout
   - More flexible data access

## Exercise

Try extending the simulation:

1. Add acceleration to particles
2. Implement particle collisions
3. Add boundary conditions

<details>
<summary>👉 Click for Exercise Hints</summary>

1. For acceleration:
   ```cpp
   // In pair version:
   cuda::std::pair<float, float> acc = cuda::std::make_pair(0.0f, -9.81f);  // gravity
   p.vel = p.vel + acc * dt;
   ```

2. For collisions:
   ```cpp
   // Check distance between particles using mdspan
   float dx = particles(i,0) - particles(j,0);
   float dy = particles(i,1) - particles(j,1);
   float dist = sqrt(dx*dx + dy*dy);
   ```

3. For boundaries:
   ```cpp
   // Reflect particles at boundaries
   if(p.pos.first > 1.0f) {
       p.pos.first = 1.0f;
       p.vel.first *= -1.0f;
   }
   ```
</details>